In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import uuid
from tqdm import tqdm
import requests
from copy import copy

In [2]:
class naver_real_estate_news_search :
    
    def __init__(self) :
        
        self.driver = webdriver.Chrome("C:/Users/dlrkd/OneDrive/Desktop/연습/크롤링/chromedriver_win32/chromedriver")
        self.url = "http://www.naver.com"
        self.driver.get(self.url)
        self.df_final = pd.DataFrame()

        
    def get_news_page(self) :
        
        search_word = "부동산"
        search_bar = WebDriverWait(self.driver, 0.3).until(EC.presence_of_element_located((By.CLASS_NAME, "input_text")))
        sleep(0.1)
        input_word = search_bar.send_keys(search_word)
        
        button = WebDriverWait(self.driver, 0.3).until(EC.presence_of_element_located((By.CLASS_NAME, "btn_submit")))
        button.click()
        
        news_button = {}
        button_bar = WebDriverWait(self.driver, 0.3).until(EC.presence_of_element_located((By.CLASS_NAME, "base"))).find_elements_by_class_name("menu")
        for button in button_bar :
            button_name = button.text
            button_href = button.find_element_by_class_name("tab")
            news_button[button_name] = button_href
            
        for text in news_button.keys() :
            if text == "뉴스" :
                news_button[text].click()
                
    def get_news_data(self) :
              
        news_area = WebDriverWait(self.driver, 0.3).until(EC.presence_of_element_located((By.CLASS_NAME, "list_news")))
        news_area_html = news_area.get_attribute("innerHTML")
        news_area_bs = BeautifulSoup(news_area_html, "html.parser")
        
        news_data = {}
        for news in news_area_bs.find_all("li", {"class" : "bx"}) :
            page = news.find("div", {"class" : "news_area"}).find("a", {"class" : "news_tit"})
            title = page.get("title").strip()
            href = page.get("href")
            news_data[title] = href
            
        news = pd.DataFrame(news_data, index= [""])
    
        return news
    
    def go_to_next_page(self, start_page, last_page) :
        search = self.get_news_page()
        
        current_pg = start_page
        last_pg = last_page
        start = ((int(current_pg)-1)*10)+1
        last = ((int(last_pg)-1)*10)+1
        
        while start <= last :
            url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%B6%80%EB%8F%99%EC%82%B0&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=19&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="+str(start)
            res = requests.get(url)
            page_bs = BeautifulSoup(res.content, "html.parser")
            news_list = page_bs.find("div", {"class" : "group_news"}).find_all("li", {"class" : "bx"})
            
            data = {}
            for news in news_list :
                page = news.find("div", {"class" : "news_area"}).find("a", {"class" : "news_tit"})
                title = page.get("title").strip()
                href = page.get("href")
                data[title] = href
            
            data_frame = pd.DataFrame(data.items())
            self.df_final = self.df_final.append(data_frame)
            
            start += 10

In [3]:
crawler = naver_real_estate_news_search()

In [4]:
crawler.go_to_next_page(2,5)

In [6]:
crawler.df_final

,0,1
0,또 다시 빅스텝 공포...“부동산 침체 장기화 피할 수 없어”,https://economist.co.kr/2022/10/12/realEstate/...
1,"“박수홍 형수 주부가 100억 부동산”...국세청장 “탈루 엄정 대응""",http://news.mk.co.kr/newsRead.php?no=901125&ye...
2,달러 강세 업은 외국 투자수요 국내 상업용 부동산에 눈독,https://economist.co.kr/2022/10/12/realEstate/...
3,"한은 총재 ""부동산 추가하락 가능성…고통 클것""",http://news.tvchosun.com/site/data/html_dir/20...
4,"""박수홍 형수, 주부인데 100억 부동산""…국세청장 ""탈루에 엄정 대응""",https://www.sedaily.com/NewsView/26CBLTDRSH
5,"[금리 3.0%] 이창용 ""부동산 가격 추가하락 가능성""",http://www.fnnews.com/news/202210121446133039
6,해양클러스터 조성에 '의정의 힘'… 합리적 부동산 정책 '방향타',http://www.kyeongin.com/main/view.php?key=2022...
7,고금리‧증시부진에 ‘어닝쇼크’…부동산PF도 ‘휘청’ [벼랑 끝 내몰린 증권사들①],https://economist.co.kr//2022/10/12/stock/stoc...
8,전남 부동산 미분양 5년간 5천558세대…전국 상위권,https://www.yna.co.kr/view/AKR2022101204600005...
9,"부동산R114-더비즈, 중개업무 경쟁력 강화 위한 업무협약",http://www.newsis.com/view/?id=NISX20221011_00...
